In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format ='{:.5f}'.format

## Задание 0

*   Новый пункт
*   Новый пункт




Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [ ]:
%%capture
!wget https://www.dropbox.com/s/axkihywa8q118rk/transactions.csv
!wget https://www.dropbox.com/s/xwbkjwik600clsh/tr_mcc_codes.csv
!wget https://www.dropbox.com/s/5deipwkmeial1ba/tr_types.csv
!wget https://www.dropbox.com/s/qc307hdkc9jdxf8/gender_train.csv

transactions = pd.read_csv('transactions.csv', nrows = 1000000)
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';')
gender_train = pd.read_csv('gender_train.csv')
tr_types = pd.read_csv('tr_types.csv',sep=';')


In [ ]:
display(tr_mcc_codes.head())

NameError: ignored

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Без использования merge!

In [ ]:

tr_type_POS_ATM = tr_types['tr_type'][( tr_types['tr_description'].str.contains("АТМ")) | ( tr_types['tr_description'].str.contains("POS"))]
transactions_sample = transactions['tr_type'].sample(1000)

POS_ATM_count = transactions_sample.isin(tr_type_POS_ATM).value_counts()[True]
print('Доля транзакций в которой содержится подстрока POS или ATM')
print(POS_ATM_count / 1000)

Доля транзакций в которой содержится подстрока POS или ATM
0.639


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
tr_type_number_count = transactions.groupby('tr_type').size().reset_index(name='counts')
display(tr_type_number_count)
top_10_tr_types = tr_type_number_count.nlargest(10, 'counts')['tr_type'].to_list()
print('Tоп-10 транзакций по частоте встречаемости')
for i in top_10_tr_types:
  description = tr_types[(tr_types['tr_type'] == i) & ( tr_types['tr_description'])].iloc[0]['tr_description']
  print(i, " ", description)

,tr_type,counts
0,1000,8
1,1010,231117
2,1030,118975
3,1100,15476
4,1110,137658
...,...,...
67,7074,505
68,7075,12
69,8100,2
70,8145,8


Tоп-10 транзакций по частоте встречаемости
1010   Покупка. POS ТУ СБ РФ
2010   Выдача наличных в АТМ Сбербанк России
7070   Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя)
1110   Покупка. POS ТУ Россия
1030   Оплата услуги. Банкоматы СБ РФ
2370   Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии)
7010   Взнос наличных через АТМ (в своем тер.банке)
7030   Перевод на карту (с карты) через АТМ (в пределах одного тер.банка)
7071   Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя)
1100   Покупка. ТУ  Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
clients_spendings = transactions[transactions['amount'] < 0].groupby('customer_id')['amount'].sum()
clients_earnings = transactions[transactions['amount'] > 0].groupby('customer_id')['amount'].sum()

top_spending_client = clients_spendings.nsmallest(1)
top_earnings_client = clients_earnings.nlargest(1)

print("Клиент с максимальной суммой приходов")
print(top_earnings_client.to_string())
print("Клиент с максимальной суммой pacходов")
print(top_spending_client.to_string())
print("Модуль разницы для этих клиентов между суммой расходов и суммой приходов")
print(np.abs(top_earnings_client.values + clients_spendings[top_earnings_client.index]).to_string())
print(np.abs(top_spending_client.values + clients_earnings[top_spending_client.index]).to_string())

Клиент с максимальной суммой приходов
customer_id
70780820   1248114886.81002
Клиент с максимальной суммой pacходов
customer_id
70780820   -1249952204.79000
Модуль разницы для этих клиентов между суммой расходов и суммой приходов
customer_id
70780820   1837317.97997
customer_id
70780820   1837317.97997


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
# для топ 10 tr_types
top_10_tr_type_avg = transactions[transactions['tr_type'].isin(top_10_tr_types)].groupby('tr_type')['amount'].mean()
top_10_tr_type_med = transactions[transactions['tr_type'].isin(top_10_tr_types)].groupby('tr_type')['amount'].median()
top_10_tr_type_avg_med = pd.concat([top_10_tr_type_avg, top_10_tr_type_med],keys=['average','median'], axis=1)
display(top_10_tr_type_avg_med)


,average,median
tr_type,,
1010,-19784.74864,-7411.52000
1030,-5320.98022,-2245.92000
1100,-44061.82726,-10188.26000
1110,-32119.33037,-11207.57000
2010,-136077.62933,-44918.32000
2370,-205418.24903,-44918.32000
7010,276391.78960,112295.79000
7030,86104.33291,13951.52000
7070,65569.83170,11319.42000


In [ ]:
# для  клиента с максимальной суммой расходов
top_spending_client_tr_types = transactions[transactions['customer_id'].isin(top_spending_client.index.to_list())]
client_tr_types_avg = top_spending_client_tr_types.groupby('tr_type')['amount'].mean()
client_tr_types_med = top_spending_client_tr_types.groupby('tr_type')['amount'].median()
client_tr_types_avg_med = pd.concat([client_tr_types_avg, client_tr_types_med],keys=['average','median'], axis=1)
display(client_tr_types_avg_med)

,average,median
tr_type,,
2010,-2941055.87094,-3368873.66000
2330,-2382398.34462,-2245915.77000
2370,-2218504.98709,-2245915.77000
6110,17562.93173,10028.01000
7020,14651.29074,10319.98500
7021,28500.67000,28500.67000
7030,14061.95524,8848.91000
7031,19210.35845,12992.62000
7034,14306.48000,14306.48000


In [ ]:
# для  клиента с максимальной суммой приходов(в нашем случае это один и тот же но они могут быть разными)
top_earnings_client_tr_types = transactions[transactions['customer_id'].isin(top_earnings_client.index.to_list())]
client_tr_types_avg = top_earnings_client_tr_types.groupby('tr_type')['amount'].mean()
client_tr_types_med = top_earnings_client_tr_types.groupby('tr_type')['amount'].median()
client_tr_types_avg_med = pd.concat([client_tr_types_avg, client_tr_types_med],keys=['average','median'], axis=1)
display(client_tr_types_avg_med)

,average,median
tr_type,,
2010,-2941055.87094,-3368873.66000
2330,-2382398.34462,-2245915.77000
2370,-2218504.98709,-2245915.77000
6110,17562.93173,10028.01000
7020,14651.29074,10319.98500
7021,28500.67000,28500.67000
7030,14061.95524,8848.91000
7031,19210.35845,12992.62000
7034,14306.48000,14306.48000


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
avg_spendings = transactions[(transactions['amount'] < 0)].groupby(['gender'])['amount'].mean()
avg_earnings = transactions[(transactions['amount'] > 0)].groupby(['gender'])['amount'].mean()

gender_spendings_diff_module = np.abs(avg_spendings[0] - avg_spendings[1])
gender_earnings_diff_module = np.abs(avg_earnings[0] - avg_earnings[1])

print("Модуль разницы между средними тратами женщин и мужчин")
print(gender_spendings_diff_module)
print("Модуль разницы между средними поступлениями у мужчин и женщин")
print(gender_earnings_diff_module)

Модуль разницы между средними тратами женщин и мужчин
32718.05492021877
Модуль разницы между средними поступлениями у мужчин и женщин
63366.57104801515


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
max_income = transactions[(transactions['amount'] > 0)].groupby(['gender','tr_type'])['amount'].max()
least_10_max_income_gender_0 = max_income[0].nsmallest(10).reset_index(name='max_income')
least_10_max_income_gender_1 = max_income[1].nsmallest(10).reset_index(name='max_income')
print('least_10_max_income для мужчин и женщин')
display(least_10_max_income_gender_0)
display(least_10_max_income_gender_1)

least_10_max_income для мужчин и женщин


,tr_type,max_income
0,4051,1122.96000
1,4110,2245.92000
2,4210,2245.92000
3,2370,2335.75000
4,4100,5041.86000
5,7075,6737.75000
6,2110,22459.16000
7,1010,24839.83000
8,2010,44918.32000
9,2210,68315.82000


,tr_type,max_income
0,2020,2245.92000
1,4110,2245.92000
2,1010,28803.87000
3,8100,64682.37000
4,6000,79280.83000
5,7041,87590.72000
6,2010,89836.63000
7,2110,112295.79000
8,2370,113194.15000
9,7015,121279.45000


In [ ]:
print('Tранзакци, которые встречаются одновременно и у мужчин, и у женщин')
merged_income = pd.merge(least_10_max_income_gender_0,least_10_max_income_gender_1, on=['tr_type'], how='inner') 
display(merged_income)

Tранзакци, которые встречаются одновременно и у мужчин, и у женщин


,tr_type,max_income_x,max_income_y
0,4110,2245.92000,2245.92000
1,2370,2335.75000,113194.15000
2,2110,22459.16000,112295.79000
3,1010,24839.83000,28803.87000
4,2010,44918.32000,89836.63000


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
mcc_code_sum_groupby_gender = transactions.groupby(['gender','mcc_code'])['amount'].sum()
display(mcc_code_sum_groupby_gender)
top_5_mcc_codes = np.abs(mcc_code_sum_groupby_gender[0] - mcc_code_sum_groupby_gender[1]).nlargest(5).index.to_list()
display(tr_mcc_codes[(tr_mcc_codes['mcc_code'].isin(top_5_mcc_codes)) & (tr_mcc_codes['mcc_description'])])


gender   mcc_code
0.00000  742          -386207.68000
         1711         -674897.69000
         1799         -261626.73000
         2741          -20746.42000
         3000       -70435914.05000
                          ...      
1.00000  8999       -28636743.64000
         9222         -988205.17000
         9311         -373271.22000
         9399        -1810820.67000
         9402           -4581.67000
Name: amount, Length: 349, dtype: float64

,mcc_code,mcc_description
22,4829,Денежные переводы
62,5511,"Легковой и грузовой транспорт — продажа, серви..."
65,5541,Станции техобслуживания
123,6010,Финансовые институты — снятие наличности вручную
124,6011,Финансовые институты — снятие наличности автом...


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
transactions[['tr_date','tr_time']] = transactions['tr_datetime'].str.split(' ',expand=True)
transactions[['tr_hour','tr_minute','tr_second']] = transactions['tr_time'].str.split(':', expand=True)

night_transactions = transactions[(transactions['tr_hour'].isin(['00','01','02','03','04','05'])) & (transactions['amount'] < 0)]
night_transactions_groupby_gender = night_transactions.groupby('gender')['amount'].count()
print('Количество транзакций с amount<0 в ночное время для мужчин и женщин')
print(night_transactions_groupby_gender)
display(transactions)

Количество транзакций с amount<0 в ночное время для мужчин и женщин
gender
0.00000    38827
1.00000    43017
Name: amount, dtype: int64


,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_date,tr_time,tr_hour,tr_minute,tr_second
0,39026145,0 10:23:26,4814,1030,-2245.92000,NaN,1.00000,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,0,10:23:26,10,23,26
1,39026145,6 07:08:31,4814,1030,-5614.79000,NaN,1.00000,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,6,07:08:31,07,08,31
2,39026145,8 07:06:10,4814,1030,-1122.96000,NaN,1.00000,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8,07:06:10,07,06,10
3,39026145,11 08:49:03,4814,1030,-2245.92000,NaN,1.00000,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,08:49:03,08,49,03
4,39026145,11 14:12:08,4814,1030,-2245.92000,NaN,1.00000,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11,14:12:08,14,12,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89000,NaN,nan,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,18,23:38:60,23,38,60
999580,6808459,206 19:00:10,3000,6000,124693.24000,NaN,nan,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,206,19:00:10,19,00,10
999581,83759123,90 22:40:57,3000,6000,218976.79000,NaN,nan,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,90,22:40:57,22,40,57
999582,89619970,104 23:39:35,3000,6000,408307.49000,NaN,nan,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,104,23:39:35,23,39,35


**Защита:**

Найдите term_id, на котором больше всего транзакций с amount<0, сделайте то же самое для обоих полов

Найдите term_id, на котором больше всего транзакций с amount>0, сделайте то же самое для обоих полов

Найдите пользователя, который совершил транзакций на самом большом количестве разных терминалов.

In [ ]:
term_id_transactions_count = transactions[transactions['amount']<0].groupby(['gender','term_id'])['amount'].count()
print("term_id, на котором больше всего транзакций с amount<0")
print(term_id_transactions_count[0].nlargest(1))
print(term_id_transactions_count[1].nlargest(1))

term_id_transactions_count = transactions[transactions['amount']>0].groupby(['gender','term_id'])['amount'].count()
print("term_id, на котором больше всего транзакций с amount>0")
print(term_id_transactions_count[0].nlargest(1))
print(term_id_transactions_count[1].nlargest(1))

print("Пользователь, который совершил транзакций на самом большом количестве разных терминалов")
most_terminals_user = transactions.groupby('customer_id')['term_id'].count().nlargest(1)
print(most_terminals_user)

term_id, на котором больше всего транзакций с amount<0
term_id
888986    1523
Name: amount, dtype: int64
term_id
888993    1269
Name: amount, dtype: int64
term_id, на котором больше всего транзакций с amount>0
term_id
888993    1475
Name: amount, dtype: int64
term_id
888987    6057
Name: amount, dtype: int64
Пользователь, который совершил транзакций на самом большом количестве разных терминалов
customer_id
70780820    50651
Name: term_id, dtype: int64
